In [1]:
# install packages
!pip install llama_index
!pip install requests
!pip install beautifulsoup4
!pip install os
!pip install logging
!pip install openai
!pip install git+https://github.com/danja/llama_index

ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


  Using cached logging-0.4.9.6.tar.gz (96 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [19 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 14, in <module>
    File "C:\anaconda3\lib\site-packages\setuptools\__init__.py", line 16, in <module>
      import setuptools.version
    File "C:\anaconda3\lib\site-packages\setuptools\version.py", line 1, in <module>
      import pkg_resources
    File "C:\anaconda3\lib\site-packages\pkg_resources\__init__.py", line 83, in <module>
      __import__('pkg_resources.extern.packaging.specifiers')
    File "C:\anaconda3\lib\site-packages\pkg_resources\_vendor\packaging\specifiers.py", line 24, in <module>
      from .utils import canonicalize_version
    File "C:\anaconda3\lib\site-packages\pkg_resources\_vendor\packaging\utils.py", line 8, in <module>
      from .tags import Tag, parse_tag
    File "C:\anaconda3\lib\site-packages\pkg_res

  Cloning https://github.com/danja/llama_index to c:\users\emeka okoye\appdata\local\temp\pip-req-build-klpqahgi
  Resolved https://github.com/danja/llama_index to commit 236a867fa58b646237aed4ab89af054bc0a5e86b
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/danja/llama_index 'C:\Users\EMEKA OKOYE\AppData\Local\Temp\pip-req-build-klpqahgi'


In [2]:
# load packages
from llama_index import download_loader
import os
import logging
from llama_index import (
    KnowledgeGraphIndex,
    ServiceContext,
)

from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import SparqlGraphStore
from llama_index.llms import OpenAI
from IPython.display import Markdown, display
from llama_index import load_index_from_storage
import os
import openai


In [3]:
############
# LLM Config
############

# My OpenAI Key
os.environ['OPENAI_API_KEY'] = "api key"
openai.api_key = os.environ["OPENAI_API_KEY"]


In [4]:

llm = OpenAI(temperature=0, model="text-davinci-002")
service_context = ServiceContext.from_defaults(llm=llm, chunk_size=512)


In [5]:
###############
# SPARQL Config
###############

# local virtuoso
ENDPOINT = 'http://linkeddata.uriburner.com/sparql'
GRAPH = 'urn:data:rdf2textbarbie'
BASE_URI = 'http://purl.org/stuff/data'

graph_store = SparqlGraphStore(
    sparql_endpoint=ENDPOINT,
    sparql_graph=GRAPH,
    sparql_base_uri=BASE_URI,
)
storage_context = StorageContext.from_defaults(graph_store=graph_store)



HTTPError: HTTP Error 406: Unacceptable

In [ ]:
# I had issues/errors with the WikipediaReader

#WikipediaReader = download_loader("WikipediaReader")
#loader = WikipediaReader()
#documents = loader.load_data(
    #pages=['Barbie (film)'], auto_suggest=False)

    
# my hack for downloading wikipedia pages   
import requests
from bs4 import BeautifulSoup
from llama_index import Document
from llama_index.schema import MetadataMode


search_term = "Barbie_(film)" 
resp = requests.get(f'https://en.wikipedia.org/wiki/{search_term}')
#print(response.content)

# Function to remove HTML tags
def remove_tags2(html):
 
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html.text, 'html.parser')

    # Extract the text without HTML tags
    text_without_html = soup.get_text()

    return text_without_html
 
 
# Print the extracted data
# print(remove_tags2(resp))
wiki_doc = remove_tags2(resp)


# convert to a Document object with metadata
documents = Document(
    text=wiki_doc,
    id_="doc_id_1",
    metadata={
        "file_name": "barbie film 2023",
        "category": "film",
        "author": "wikipedia"    
    },
)


In [ ]:
# knowledge graph index
kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    service_context=service_context,
    max_triplets_per_chunk=10,
    sparql_endpoint=ENDPOINT,
    sparql_graph=GRAPH,
    sparql_base_uri=BASE_URI,
    include_embeddings=True,
)

# query engine
kg_rag_query_engine = kg_index.as_query_engine(
    include_text=False,
    retriever_mode="keyword",
    response_mode="tree_summarize",
)



In [ ]:
# Question
response_graph_rag = kg_rag_query_engine.query(
    "In the movie, what does Barbie think about?")

# print(str(response_graph_rag))
display(Markdown(f"<b>{response_graph_rag}</b>"))
